# Chapter 10: PyTorch, calculus, and gradient descent

Enough playing around. It's time to get serious. 😰

This programming assignment is our chance to implement the gradient-based optimization algorithms that we studied [in class](https://mml.johnmyersmath.com/stats-book/chapters/11-optim.html#). These implementations will use the differentiation engine built into the Python library called [PyTorch](https://pytorch.org/). This library is used in many cutting-edge, state-of-the-art machine learning models.

PyTorch comes with its own new type of data structure, called _PyTorch tensors_, which you should think of as NumPy arrays with lots of extra functionality. Therefore, before we get busy with coding our gradient descent algorithms, we need to learn a bit about tensors. After that, you will learn how to use PyTorch to compute many of the objects that we discussed in class, like gradient vectors, Hessian matrices, eigenvalues and eigenvectors, condition numbers, _etc_.

The second half of the assignment centers on gradient descent algorithms, both the basic non-stochastic version and the stochastic version. Starting from code templates that I provide, you will implement both of these algorithms and get a chance to experiment with them, helping build your intuition for parameter selection, diagnosing convergence, and other things.

Have fun. 😈

## Directions

1. The programming assignment is organized into sequences of short problems. You can see the structure of the programming assignment by opening the "Table of Contents" along the left side of the notebook (if you are using Google Colab or Jupyter Lab).

2. Do not add any cells of your own to the notebook, or delete any existing cells (either code or markdown).

## Submission instructions

1. Once you have finished entering all your solutions, you will want to rerun all cells from scratch to ensure that everything works OK. To do this in Google Colab, click "Runtime -> Restart and run all" along the top of the notebook.

2. Now scroll back through your notebook and make sure that all code cells ran properly.

3. If everything looks OK, save your assignment and upload the `.ipynb` file at the provided link on the course <a href="https://github.com/jmyers7/stats-book-materials">GitHub repo</a>. Late submissions are not accepted.

4. You may submit multiple times, but I will only grade your last submission.

## PyTorch basics

The first half of this programming assignment centers on the basics of PyTorch tensors, and using them to compute various objects of interest in calculus.

### Tensors

As I mentioned in the introduction, you should think of a PyTorch tensor like a NumPy array, but with lots of extra functionality. In particular, PyTorch tensors are specifically built to be fed into the PyTorch differentiation engine, as we will see later.

To get started, let's first import PyTorch by calling `import torch`. Then, let's define a $2$-dimensional tensor $\mathbf{A}$ (i.e., a matrix) by calling the `torch.tensor` constructor:

In [ ]:
import torch

A = torch.tensor([[1, 2], [3, 4]])
A

Let's check its shape (i.e., size) and dimension by accessing the `shape` attribute and calling the `dim` method (just like in NumPy):

In [ ]:
print(A.shape)
print(A.dim())

Most of the basic operations that you can do with NumPy arrays, you can also do with PyTorch tensors. For example, let's multiply two $2$-dimensional tensors (i.e., matrices) using the `@` operator:

In [ ]:
B = torch.tensor([[0, 1, -1], [3, 4, 5]])
A @ B

Vectors are $1$-dimensional tensors, like this:

In [ ]:
v = torch.tensor([-1, 3])
v

PyTorch does not express a preference for either column or row vectors, and instead uses $1$-dimensional tensors in their place. This is what `v` is. Let's check its shape and dimension:

In [ ]:
print(v.shape)
print(v.dim())

Note that the shape of `v` is just $2$ (as a $1$-dimensional tensor), not $2\times 1$ or $1\times 2$, as you would expect for either a column or a row vector.  You can _force_ `v` to be a column vector by reshaping it:

In [ ]:
v.reshape(2, 1)

Or, you can force `v` to be row vector by reshaping in the other direction:

In [ ]:
v.reshape(1, 2)

One place where using $1$-dimensional tensors in place of column and row vectors can be confusing is in the lack of transpose in certain operations involving vectors and matrices. For example, the Euclidean inner product between two (column) vectors in $\mathbb{R}^n$ is given by the matrix product $\mathbf{v}^\intercal \mathbf{w}$. This assumes that $\mathbf{v}$ and $\mathbf{w}$ are both column vectors, of size $n\times 1$. But if these vectors are implemented as $1$-dimensional tensors in PyTorch, the tranpose is not needed:

In [ ]:
v = torch.tensor([1, 2, 4])
w = torch.tensor([0, -1, 2])
v @ w

Likewise, if $\mathbf{v}$ is $n\times 1$ and $\mathbf{A}$ is $n\times m$, then we may form the product $\mathbf{v}^\intercal \mathbf{A}$ to obtain a $1\times m$ row vector. But if $\mathbf{v}$ is a $1$-dimensional tensor, you don't need the transpose:

In [ ]:
A = torch.tensor([[1, 2], [0, -1], [3, 1]])   # 3x2 matrix
v @ A

#### Problem 1 --- Basic tensor algebra

Consider the matrices given by

$$
\mathbf{A} = \begin{bmatrix}
1 & 2 & 3 \\ 0 & -1 & 4
\end{bmatrix}, \quad
\mathbf{B} = \begin{bmatrix}
1 & 3 \\
-1 & 10 \\
0 & 5
\end{bmatrix}, \quad
\mathbf{C} = \begin{bmatrix}
0 & -2 \\
3 & -4 \\
0 & 7
\end{bmatrix}
$$

In the next code cell, implement all three as $2$-dimensional tensors, calling them `A`, `B`, and `C`.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Now, in the next code cell, compute the product $\mathbf{A} \mathbf{B}$ and the linear combination $2\mathbf{B}+3\mathbf{C}$. Print your answers so that I can see them both.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Suppose that we have the vector $\mathbf{v}^\intercal = \begin{bmatrix} 2 & 3 \end{bmatrix}$. In the next code cell, compute the product $\mathbf{v}^\intercal \mathbf{A}$.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



### Partial derivatives and gradients

Consider the function

$$
f:\mathbb{R}^3 \to \mathbb{R}, \quad f(x,y,z) = 6(x^2+y) z^3.
$$

As you may easily compute by hand, we have

\begin{align*}
\frac{\partial f}{\partial x}(x, y, z) &= 12xz^3 &\Rightarrow \frac{\partial f}{\partial x}(1, -2, 2) &= 96, \\
\frac{\partial f}{\partial y}(x, y, z) &= 6z^3 &\Rightarrow \frac{\partial f}{\partial y}(1, -2, 2) &= 48, \\
\frac{\partial f}{\partial z}(x, y, z) &= 18(x^2+y)z^2 &\Rightarrow \frac{\partial f}{\partial z}(1, -2, 2) &= -72.
\end{align*}

These partial derivatives may be computed in PyTorch as follows. Run the cell, and then read the description of each step below.

In [ ]:
# step 1
def f(x, y, z):
  return 6 * (x ** 2 + y) * z ** 3

# step 2
x = torch.tensor([1.], requires_grad=True)
y = torch.tensor([-2.], requires_grad=True)
z = torch.tensor([2.], requires_grad=True)

# step 3
objective = f(x, y, z)

# step 4
objective.backward()

# step 5
print(x.grad)
print(y.grad)
print(z.grad)

Here's the explanation for each step:

1. First, define the function $f$ as a Python function.
2. Define three tensors representing the input $(x,y,z) = (1, -2, 2)$ to the partial derivatives. The tensors must have a floating point data type, so  integers must be followed by a trailing decimal point. We must also set `requires_grad=True` in order to access the partial derivatives later.
3. Compute the functional value $f(1, -2, 2)$, storing it in the variable `objective`.
4. Compute the partial derivatives by calling the `backward` method on `objective`.
5. Access the partial derivatives via the `grad` attributes on the `x`, `y`, and `z` tensors.

The reason that the method (in step 4) for computing derivatives is called `backward` comes from terminology in deep learning (in particular, the so-called "backward" pass during the [backpropagation algorithm](https://en.wikipedia.org/wiki/Backpropagation)).

Alternatively, we may assemble the individual inputs $x$, $y$, and $z$ together into an input vector:

$$
\boldsymbol{\theta}^\intercal = \begin{bmatrix} x & y & z \end{bmatrix}.
$$

Then, we may redefine $f$ as a function of $\boldsymbol{\theta}$ and compute the gradient vector $\nabla f(1, -2, 2)$ in one shot. Run the next code cell to see how this is done.

In [ ]:
def f(theta):
  x = theta[0]
  y = theta[1]
  z = theta[2]
  return 6 * (x ** 2 + y) * z ** 3

theta = torch.tensor([1., -2., 2.], requires_grad=True)

objective = f(theta)

objective.backward()

theta.grad

#### Problem 2 --- Computing gradients

In the next code cell, I have implemented an objective function of the form $J: \mathbb{R}^2 \to \mathbb{R}$ for you, as it is somewhat complicated. I wanted to give you a function with interesting contours! So, all you need to do is execute this cell.

In [ ]:
from math import sqrt

def J(theta):
   theta1 = (sqrt(2) / 2) * (theta[0] - theta[1])
   theta2 = (sqrt(2) / 2) * (theta[0] + theta[1])
   return (theta1 ** 2 + 10 * theta2 ** 2) * ((theta1 - 1) ** 2 + 10 * (theta2 - 1) ** 2) * ((theta1 + 1) ** 2 + 10 * (theta2 - 2) ** 2)

To get an idea of the shape of the graph of $J$, let's look at its contours:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x, y = np.mgrid[-1.2:2.8:0.01, -1.2:2.8:0.01]
grid = np.dstack((x, y))
z = np.apply_along_axis(J, axis=-1, arr=grid)

contour = plt.contour(x, y, z, levels=np.arange(0, 1000, 100), cmap='plasma')
plt.clabel(contour, fontsize=8)
plt.xlabel('$\\theta_1$')
plt.ylabel('$\\theta_2$')
plt.gca().set_aspect('equal')
plt.tight_layout()

In the next code cell, you will compute the gradient vector $\nabla J(1, 1)$ by using the template I provided above. Store the functional value $J(1, 1)$ in the variable `objective`, and store your gradient in the variable `grad`. Be sure to print `grad`, so that I can see it.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Now, normalize the gradient so that it has magnitude $1$, and multiply it by a negative to reverse its direction. Save your answer in the variable `unit_neg_grad`, and be sure to print it out. (You'll have to look up on your own how to normalize a $1$-dimensional tensor in PyTorch.)

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Let's check your answer. When you execute the next cell, you should see that your unit negative gradient vector is orthogonal to the contour passing through its tail and that it is pointing downhill.

In [ ]:
theta = torch.tensor([1., 1.], requires_grad=True)
objective = J(theta)
objective.backward()
unit_neg_grad = -theta.grad / torch.linalg.norm(theta.grad)

plt.contour(x, y, z, levels=np.arange(0, 1000, 100), cmap='plasma')
plt.contour(x, y, z, levels=[objective.detach().numpy()])
plt.quiver(*theta.detach().numpy(), *unit_neg_grad, angles='xy', scale_units='xy', scale=1, zorder=3)
plt.xlabel('$\\theta_1$')
plt.ylabel('$\\theta_2$')
plt.gca().set_aspect('equal')
plt.tight_layout()

In the next code cell, compute the minimum rate of change of $J$ at the point $\boldsymbol{\theta} = (1, 1)$. (You'll first need to remember from class how we find this rate.)

In [ ]:
# ENTER YOUR CODE IN THIS CELL



### Hessian matrices

Let's return to our function

$$
f:\mathbb{R}^3 \to \mathbb{R}, \quad f(x,y,z) = 6(x^2+y) z^3,
$$

discussed at the beginning of the previous section. We compute its Hessian matrix by hand as:

$$
\nabla^2 f(x, y, z) = \begin{bmatrix}
12 z^3 & 0 & 36xz^2 \\
0 & 0 & 18z^2 \\
36xz^2 & 18z^2 & 36(x^2+y)z
\end{bmatrix}.
$$

Thus,

$$
\nabla^2 f(1, -2, 2) = \begin{bmatrix}
96 & 0 & 144 \\ 0 & 0 & 72 \\ 144 & 72 & -72
\end{bmatrix}.
$$

The next code cell shows how these computations are implemented in PyTorch:

In [ ]:
from torch.autograd.functional import hessian

theta = torch.tensor([1., -2., 2.])
hess = hessian(f, theta)
hess

Let's get the eigenvalues and eigenvectors of the Hessian matrix:

In [ ]:
# hide warnings about casting complex numbers to floats. you only need to import this once.
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

evals, evecs = torch.linalg.eig(hess)
evals = evals.float()
evecs = evecs.float()
print(evals)
print(evecs)

Notice that `evals` is a $1$-dimensional tensor containing the eigenvalues, while `evecs` is a $2$-dimensional tensor whose columns are the corresponding (unit) eigenvectors.

Recall from class that the eigenvalues are the curvatures of the graph of $f$ at the point $(1, -2,2)$ in the directions specified by the corresponding eigenvectors. Let's double check that this is true, by using the formula

$$
f''_\mathbf{v}(1,-2,2) = \mathbf{v}^\intercal \left(\nabla^2 f(1, -2, 2)\right) \mathbf{v}
$$

that we learned in class. Supposing that $\mathbf{v}$ is the second eigenvector, here's the code:

In [ ]:
v = evecs[:, 1]
v @ hess @ v

Notice that, indeed, the curvature is given by the corresponding eigenvalue.

#### Problem 3 --- Computing Hessian matrices

We return to the function $J$ from Problem 2. In the next code cell, compute the Hessian matrix $\nabla^2 J(\sqrt{2},0)$. Save your answer into the variable `hess`, and be sure to print it out.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Now, compute the eigenvalues and eigenvectors of the Hessian matrix. Be sure to convert them to floats as I did in the example above, and save them into the variables `evals` and `evecs`. Print them out.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Let's check your answer. Assuming you coded everything correctly, when you execute the next code cell you should see your eigenvectors with their tails at the point $(\sqrt{2},0)$ on the contour plot of the function $J$.

In [ ]:
x_evecs = evecs[0, :]
y_evecs = evecs[1, :]

x, y = np.mgrid[-1.2:2.8:0.01, -1.2:2.8:0.01]
grid = np.dstack((x, y))
z = np.apply_along_axis(J, axis=-1, arr=grid)

plt.contour(x, y, z, levels=np.arange(0, 1000, 100), cmap='plasma')
plt.contour(x, y, z, levels=[objective.detach().numpy()])
plt.quiver([sqrt(2)] * 2, [0] * 2, x_evecs, y_evecs, angles='xy', scale_units='xy', scale=1, zorder=3)
plt.xlabel('$\\theta_1$')
plt.ylabel('$\\theta_2$')
plt.gca().set_aspect('equal')
plt.tight_layout()

Notice that the eigenvectors are orthogonal to each other (since the Hessian matrix is symmetric). The eigenvector pointing in the northwesterly direction has an eigenvalue of $\approx 308$, while the eigenvector pointing in the southwesterly direction has an eigenvalue of $\approx 3080$. These are curvatures!

Even though you already know the condition number of the Hessian matrix $\nabla^2 J(\sqrt{2},0)$ because you already computed its spectrum, in the next code cell I want you to hunt for a way to compute the condition number directly using a method from PyTorch. Once you find the appropriate method, compute the condition number, saving it into the variable `condition_num`, and then print it out.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



## Optimization algorithms

We will now work towards coding implementations of gradient descent and stochastic gradient descent, from scratch. While the algorithms are relatively straightforward, coding them can be a challenge for new coders. This is mainly because we want the algorithms to output more than just a single point estimate for the minimizer, but also the entire sequence of approximations produced by the algorithm along with all the associated objective values, and handling all the logistics of tracking these things is a bit of a headache. But don't worry!---I will help you along by providing templates for the algorithms, and your only job is to fill in the missing fragments of code.

The focus in this assignment is making sure we understand the internal workings of the basic gradient descent algorithms by coding them by hand. In the real world, unless you're a researcher inventing your own optimization algorithms, you would use algorithms that have been pre-coded for you, like those that come bundled with PyTorch. (Click [here](https://pytorch.org/docs/stable/optim.html#algorithms) to see the list of optimization algorithms in PyTorch.)


### Gradient descent

We begin with the basic multi-variable gradient descent algorithm described in the textbook [here](https://mml.johnmyersmath.com/stats-book/chapters/11-optim.html#gd-alg). Make sure to walk through the textbook description multiple times, in detail, before attempting this portion of the assignment.

#### Problem 4 --- Implementing gradient descent

In the next code cell, I have coded a template for the gradient descent algorithm. The template is in the form of a function, called `GD`, that we will call whenever we need the algorithm. Your job is to complete the template to get a functioning implementation. But before doing that, let's discuss the call signature of the `GD` function:

`GD(J, theta0, lr, num_steps, decay_rate=0)`

The parameters are:

* `J`, for the objective function $J$
* `theta0` for the intial guess $\boldsymbol{\theta}_0$
* `lr` for the learning rate $\alpha$
* `num_steps` for the number $N$ of gradient steps
* `decay_rate` for the decay rate $\beta$, set to a default value of `0`

I recommend that you read through _all_ of the code in the template to get a sense of what's going on, but you only need to fill the four `None` placeholders in the code labeled block 1 through block 4. Here are the descriptions of these four blocks:

1. Begin the main `for` loop.

2. Compute the gradient of the objective with resepect to $\boldsymbol{\theta}$.

3. Take a gradient step according to the update rule.

4. Compute an updated objective value using the new $\boldsymbol{\theta}$.

You need to fill in the missing code using these descriptions! Be very careful to **only** replace the `None`'s that are marked block 1 through block 4. You will see other `None`'s in the class `GD_output`, but do **not** replace these!



In [ ]:
# ENTER YOUR CODE IN THIS CELL

from dataclasses import dataclass

# do NOT replace the `None`'s in this class!!!
@dataclass
class GD_output:
    thetas: torch.Tensor = None
    objectives: torch.Tensor = None
    per_step_objectives: torch.Tensor = None
    per_epoch_objectives: torch.Tensor = None
    epoch_step_nums: torch.Tensor = None
    lr: float = None
    num_steps: int = None
    decay_rate: float = None
    batch_size: int = None
    num_epochs: int = None
    max_steps: int = None

    def __str__(self):
      if self.per_step_objectives == None:
        return f'learning rate: {self.lr}, gradient steps: {self.num_steps}, decay rate: {self.decay_rate}' \
        f'\n\nthetas:\n{self.thetas}\n\nobjectives:\n{self.objectives}'
      else:
        return f'learning rate: {self.lr}, number of epochs: {self.num_epochs}, batch size: {self.batch_size}, decay rate: {self.decay_rate}' \
        f'\n\nthetas:\n{self.thetas}\n\nper-step objectives:\n{self.per_step_objectives}' \
        f'\n\nper-epoch mean objectives:\n{self.per_epoch_objectives}' \
        f'\n\nepochs completed on the follow gradient steps:\n{self.epoch_step_nums}'


# begin the gradient descent function
def GD(J, theta0, lr, num_steps, decay_rate=0):

    theta = theta0.clone().requires_grad_()
    objective = J(theta)
    objectives = [objective.detach()]
    thetas = [theta.detach().clone()]

    # block 1
    None        # <-- replace `None` with your own code

        # block 2
        None    # <-- replace `None` with your own code

        # block 3
        with torch.no_grad():
            theta -= None         # <-- replace `None` with your own code

        # block 4
        None                      # <-- replace `None` with your own code

        objectives.append(objective.detach())
        thetas.append(theta.detach().clone())
        theta.grad.zero_()

    thetas = torch.row_stack(thetas)
    objectives = torch.stack(objectives)
    output = GD_output(thetas=thetas,
                       objectives=objectives,
                       lr=lr,
                       num_steps=num_steps,
                       decay_rate=decay_rate)
    return output

To check your work, we will return to the objective function $J$ from Problems 2 and 3 above, and use the gradient descent algorithm to find the local minimum at $(\sqrt{2},0)$. Execute the next cell, which runs the algorithm with learning rate $\alpha = 0.001$ over $N=25$ gradient steps beginning from the initial value $\boldsymbol{\theta}_0=(1,1)$.

In [ ]:
# gradient descent parameters
theta0 = torch.tensor([1., 1.])
alpha = 0.001
N = 25

# run the algorithm, collect the output into `gd_output`
gd_output = GD(J=J, theta0=theta0, lr=alpha, num_steps=N)

Our implementation of the `GD` function returns an object of the custom class `GD_output`, which holds the approximations

$$
\boldsymbol{\theta}_0,\boldsymbol{\theta}_1,\ldots,\boldsymbol{\theta}_N
$$

along with the objective values

$$
J(\boldsymbol{\theta}_0),J(\boldsymbol{\theta}_1),\ldots,J(\boldsymbol{\theta}_N).
$$

Let's check the contents of the output of the algorithm by printing the `gd_output` object:

In [ ]:
print(gd_output)

If your code is correct, then you should see sensible output. The rows of the first tensor in the printout are the $\boldsymbol{\theta}$'s generated by the algorithm, while the second tensor contains the associated objective values.

But plots would be much more informative! Since we will need to plot the outputs from multiple runs of the algorithm, let's code a utility function for plotting. Execute the next cell.

In [ ]:
def plot_gd_output(gd_output):
  fig, axes = plt.subplots(ncols=2, figsize=(9, 4))

  # plot the trace of the algorithm
  axes[0].contour(x, y, z, levels=np.arange(0, 1000, 100), cmap='plasma')
  axes[0].plot(gd_output.thetas[:, 0], gd_output.thetas[:, 1])
  axes[0].scatter(gd_output.thetas[:, 0], gd_output.thetas[:, 1], s=30, zorder=3)
  axes[0].set_xlabel('$\\theta_1$')
  axes[0].set_ylabel('$\\theta_2$')
  axes[0].set_aspect('equal')

  # plot objective versus gradient steps
  axes[1].plot(range(N + 1), gd_output.objectives)
  axes[1].scatter(range(N + 1), gd_output.objectives)
  axes[1].set_xlabel('gradient steps')
  axes[1].set_ylabel('objective')

  fig.suptitle(f'$\\alpha={gd_output.lr}$, $\\beta={gd_output.decay_rate}$, $N={gd_output.num_steps}$')
  plt.tight_layout()

Now, using our helper function, let's check your code for the gradient descent algorithm. Execute the next cell.

In [ ]:
plot_gd_output(gd_output)

You should see that the algorithm finds the general neighborhood of the minimizer at $(\sqrt{2},0)$, but then oscillates back and forth (mostly in the direction of maximum curvature) and never fully converges.

#### Problem 5 --- Obtaining convergence

Let's alter some of the parameters for the gradient descent algorithm in order to obtain convergence. In the next code cell, choose a suitable value for the decay rate $\beta$ so that the algorithm converges to the local minimizer.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

# gradient descent parameters
theta0 = torch.tensor([1., 1.])
alpha = 0.001
N = 25
beta = None         # <-- replace `None` with your own code

gd_output = None    # <-- replace `None` with your own code

# plot the output
None                # <-- replace `None` with your own code

Rather than increasing the decay rate in order to encourange convergence and dampen oscillations, we saw in class (in [this](https://mml.johnmyersmath.com/stats-book/chapters/11-optim.html#quadratic-conv-thm) theorem) that we can set the learning rate to the reciprocal mean curvature of the graph of $J$ at the local minimum. In the next code cell, set the learning rate $\alpha$ to this value and run the algorithm. (Use $\beta=0$.)

In [ ]:
# ENTER YOUR CODE IN THIS CELL

# gradient descent parameters
theta0 = torch.tensor([1., 1.])
alpha = None        # <-- replace `None` with your own code
N = 25

gd_output = None    # <-- replace `None` with your own code

# plot the output
None                # <-- replace `None` with your own code

Lovely convergence!

In the next code cell, we print the last $\boldsymbol{\theta}$ produced by the algorithm as our final approximation to the minimizer.

In [ ]:
print('approximate minimizer found by gradient descent:', gd_output.thetas[-1].tolist())
print('true minimizer:                                 ', [np.sqrt(2), 0])


### Stochastic gradient descent

We now turn toward the stochastic gradient descent algorithm, as described in the textbook [here](https://mml.johnmyersmath.com/stats-book/chapters/11-optim.html#sgd-alg). Of course, the biggest difference between this version of gradient descent and the non-stochastic one is that the data must be (randomly) partitioned into mini-batches over each epoch. To do this, we will use a built-in utility from PyTorch from the DataLoader class (see [here](https://pytorch.org/docs/stable/data.html#)). Other than that, the basic pieces of the stochastic version of the algorithm are more or less the same as the non-stochastic one.

But as we talked about in class, to monitor convergence of the stochastic version of the algorithm, it is convenient to track _both_ the per-step objective values and also the mean per-epoch objective values. This adds a level of complexity to the implementation that is not present in the non-stochastic version.

#### Problem 6 --- Implementing stochastic gradient descent

The format of this problem is exactly the same as Problem 4. But this time, we will code a function, called `SGD`, which implements _stochastic_ gradient descent. Its call signature will be:

`SGD(g, theta0, X, lr, batch_size, num_epochs, decay_rate=0, max_steps=-1, shuffle=True, random_state=None)`

The parameters are:

* `g` for the funtion described in the textbook [here](https://mml.johnmyersmath.com/stats-book/chapters/11-optim.html#sgd-alg)

* `theta0` for the initial $\boldsymbol{\theta}_0$

* `X` for a $2$-dimensional tensor (i.e., matrix) whose rows are the dataset $\mathbf{x}_1,\mathbf{x}_2\ldots,\mathbf{x}_m$

* `lr` for the learning rate $\alpha$

* `batch_size` for the mini-batch size $k$

* `num_epochs` for the number of epochs $N$

* `decay_rate` for the decay rate $\beta$

* `max_steps` for a parameter that halts the algorithm at a specified number of gradient steps

There are two more parameters, `shuffle` and `random_state`, that you don't need to worry about.

As the stochastic version of the algorithm is more complex than the basic version, so too will our code be more complex. The template below contains the basic skeleton of the code. As in Problem 4, your job is only to replace five `None` placeholders in the code labeled block 1 through block 4. Here are the descriptions of these blocks:

1. Begin the outer and inner `for` loops through the epochs and mini-batches.

2. Compute the gradient of the objective with respect to $\boldsymbol{\theta}$.

3. Take a gradient step using the update rule.

4. Compute the new objective value with the updated $\boldsymbol{\theta}$.

Using these descriptions, fill in your code!

In [ ]:
# ENTER YOUR CODE IN THIS CELL

from torch.utils.data import DataLoader

def SGD(g, theta0, X, lr, batch_size, num_epochs, decay_rate=0, max_steps=-1, shuffle=True, random_state=42):

    if random_state != None:
        torch.manual_seed(random_state)
    data_loader = DataLoader(dataset=X, batch_size=batch_size, shuffle=shuffle)
    num_mini_batches = len(data_loader)

    theta = theta0.clone().requires_grad_()
    per_step_objectives = []
    per_epoch_objectives = []
    thetas = [theta.detach().clone()]

    first_step = True
    s = 0
    epoch_step_nums = [0]

    # block 1
    None        # <-- replace `None` with your own code

        for i, mini_batch in enumerate(data_loader):

            if first_step:
                objective = g(mini_batch, theta).mean()
                per_step_objectives.append(objective.detach())
                per_epoch_objectives.append(objective.detach())
                first_step = False

            # block 2
            None        # <-- replace `None` with your own code

            # block 3
            None        # <-- replace `None` with your own code
            None        # <-- replace `None` with your own code

            # block 4
            None        # <-- replace `None` with your own code

            per_step_objectives.append(objective.detach())
            thetas.append(theta.detach().clone())
            theta.grad.zero_()

            complete_epoch = True if i + 1 == num_mini_batches else False
            s += 1
            if s == max_steps:
                break

        if complete_epoch:
            epoch_step_nums.append(s)

        epoch_objective = torch.tensor(per_step_objectives[num_mini_batches * t + 1:])
        per_epoch_objectives.append(epoch_objective.mean())

        if s == max_steps:
            break

    thetas = torch.row_stack(thetas)
    per_step_objectives = torch.stack(per_step_objectives)
    per_epoch_objectives = torch.stack(per_epoch_objectives)
    epoch_step_nums = torch.tensor(epoch_step_nums)
    output = GD_output(thetas=thetas,
                       per_step_objectives=per_step_objectives,
                       per_epoch_objectives=per_epoch_objectives,
                       epoch_step_nums=epoch_step_nums,
                       lr=lr,
                       num_epochs=num_epochs,
                       decay_rate=decay_rate,
                       batch_size=batch_size,
                       max_steps=max_steps)
    return output

In order to check whether your code is correct, we need a dataset and stochastic objective function. To get the dataset, run the next code cell, which generates the same dataset drawn from a 2-dimensional normal distribution used in the [textbook](https://mml.johnmyersmath.com/stats-book/chapters/11-optim.html#stochastic-gradient-descent):

In [ ]:
import seaborn as sns
from torch.distributions.multivariate_normal import MultivariateNormal

torch.manual_seed(42)
X = MultivariateNormal(loc=torch.zeros(2), covariance_matrix=torch.eye(2)).sample(sample_shape=(1024,))

sns.scatterplot(x=X[:, 0], y=X[:, 1])
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.gcf().set_size_inches(w=5, h=3)
plt.tight_layout()

Notice that the dataset is loaded into a $2$-dimensional tensor `X` (i.e., a matrix).

Now, in the next code cell, I want you to implement the function

$$
g:\mathbb{R}^4 \to \mathbb{R}, \quad g(\mathbf{x};\boldsymbol{\theta}) = \frac{1}{2}\left[ (x_1 - \theta_1)^2 + 4(x_2 - \theta_2)^2\right]
$$

along with the associated stochastic objective function

$$
J:\mathbb{R}^2 \to \mathbb{R}, \quad J(\boldsymbol{\theta}) = \frac{1}{m} \sum_{i=1}^m g(\mathbf{x}_i;\boldsymbol{\theta}).
$$

Once you enter your code and run the cell, you should see a plot of the contours of $J$.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

def g(X, theta):
    x1 = X[:, 0]
    x2 = X[:, 1]
    theta1 = theta[0]
    theta2 = theta[1]
    return None         # <-- replace `None` with your own code

def J(theta):
    return None         # <-- replace `None` with your own code

x, y = np.mgrid[-2:2:0.05, -2:2:0.05]
grid = np.dstack([x, y])
z = np.apply_along_axis(J, axis=-1, arr=grid)

contour = plt.contour(x, y, z, levels=np.arange(0, 10, 0.5), cmap='plasma')
plt.clabel(contour)
plt.xlabel('$\\theta_1$')
plt.ylabel('$\\theta_2$')
plt.gca().set_aspect('equal')

Now that we have a dataset, a function $g$, and the stochastic objective function $J$, we may run our stochastic gradient descent algorithm. Do so in the following cell, by filling in the `None` placeholder.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

# stochastic gradient descent parameters
theta0 = torch.tensor([1.5, 1.5])
alpha = 0.3
k = 128
N = 4

# run the algorithm, collect the output into `gd_output`
gd_output = None        # <-- replace `None` with your own code

Let's check the output. Run the following cell.

In [ ]:
print(gd_output)

This printout is nice, but a visualization would be better. Run the next cell, which implements a helper function for plotting the output of the algorithm.

In [ ]:
def plot_sgd_output(gd_output):
  fig, axes = plt.subplots(ncols=2, figsize=(9, 4))

  # plot the trace of the algorithm
  axes[0].contour(x, y, z, levels=np.arange(0, 10, 0.25), cmap='plasma')
  axes[0].plot(gd_output.thetas[:, 0], gd_output.thetas[:, 1])
  axes[0].scatter(gd_output.thetas[:, 0], gd_output.thetas[:, 1], s=30, zorder=3)
  axes[0].set_xlabel('$\\theta_1$')
  axes[0].set_ylabel('$\\theta_2$')
  axes[0].set_aspect('equal')

  # plot objective versus gradient steps
  axes[1].plot(range(len(gd_output.per_step_objectives)), gd_output.per_step_objectives, alpha=0.25, label='objective per step')
  axes[1].scatter(range(len(gd_output.per_step_objectives)), gd_output.per_step_objectives, alpha=0.25)
  axes[1].plot(gd_output.epoch_step_nums, gd_output.per_epoch_objectives, label='mean objective per epoch')
  axes[1].scatter(gd_output.epoch_step_nums, gd_output.per_epoch_objectives, zorder=3)
  axes[1].set_xlabel('gradient steps')
  axes[1].set_ylabel('objective')
  axes[1].legend()

  fig.suptitle(f'$\\alpha={gd_output.lr}$, $\\beta={gd_output.decay_rate}$, $k={gd_output.batch_size}$, $N={gd_output.num_epochs}$')
  plt.tight_layout()

Using the helper function, in the next code cell, pass in the output of the algorithm to obtain a visualization.

In [ ]:
# ENTER YOUR CODE IN THIS CELL



#### Problem 7 --- Experimenting with convergence

In the next code cell, try selecting different values for the size $k$ of the mini-batches and plotting the results. What happens if you select $k=1$, so that the algorithm takes a gradient step per data point? What happens if you choose $k=1{,}024$, so that the algorithm turns into [batch gradient descent](https://mml.johnmyersmath.com/stats-book/chapters/11-optim.html#batch-gd-def)? What happens for values in between these two extremes? For your final answer, select your favorite mini-batch size out of all the ones you try. 😁

In [ ]:
# ENTER YOUR CODE IN THIS CELL

# stochastic gradient descent parameters
theta0 = torch.tensor([1.5, 1.5])
alpha = 0.3
k = None            # <-- replace `None` with your own code
N = 8

# run the algorithm, collect the output into `gd_output`
gd_output = None    # <-- replace `None` with your own code

# plot the results
None                # <-- replace `None` with your own code